## Modelo criado para a competição da 2ºFase do trabalho da UC de Dados e Aprendizagem Automática

### OBJETIVO -  PREVER A quantidade de energia injectada na rede eléctrica QUE SE ENCONTRA NA COLUNA Injeção na rede (kWh) DO DATASET DE ENERGIA 

### Imports de bicilotecas necessárias para desenvolver o modelo 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

### Leitura dos dados relativos a 09/21-12/21

In [2]:
df_meteo = pd.read_csv("../../datasets/parte2/treino/meteo_202109-202112.csv")
df_energia = pd.read_csv("../../datasets/parte2/treino/energia_202109-202112.csv",na_filter=False)

### Passagem da coluna dt_iso para as duas colunas Data e Hora para depois podermos dar merge dos dois datasets

In [3]:
df_meteo['dt_iso'] = pd.to_datetime(df_meteo['dt_iso'].str.replace(' UTC', ''), format='%Y-%m-%d %H:%M:%S %z')
df_meteo['Data'] = df_meteo['dt_iso'].dt.strftime('%Y-%m-%d')
df_meteo['Hora'] = df_meteo['dt_iso'].dt.strftime('%H')
df_meteo['Hora'] = df_meteo['Hora'].astype(int)

### Eliminar a coluna 'dt_iso' do dataset df_meteo pois os dados relevantes dessa coluna estão na nova coluna 'Data' e 'Hora' 

In [4]:
df_meteo = df_meteo.drop('dt_iso',axis = 1)

### Limpeza das colunas que não acrescentam muita informação relativa ao dataset  

In [5]:
valoresDif = df_meteo['city_name'].value_counts()

In [6]:
df_meteo = df_meteo.drop('city_name', axis = 1)
df_meteo = df_meteo.drop('sea_level', axis = 1) # Como os valores são sempre NaN e não são úteis para o contexto do problema podem ser retirados
df_meteo = df_meteo.drop('grnd_level', axis = 1) # Como os valores são sempre NaN e não são úteis para o contexto do problema podem ser retirados
df_meteo = df_meteo.drop('dt', axis = 1) # Como os valores não acrescentam informação relevante para o conexto do problema podem ser retirados 

## Tratamento de dados do dataset df_energia

### Alteração da coluna 'Hora' do dataset df_energia para o tipo int

In [7]:
df_energia['Hora'] = df_energia['Hora'].astype(int)

## Junção dos datasets df_meteo e df_energia de 09/2021 - 12/2021 para a utilização de Random Forests

### Podemos agora juntar os dois datasets pelas colunas 'Data' e 'Hora' presentes em ambos os datasets

In [8]:
df_2021= pd.merge(df_meteo, df_energia, on=['Data','Hora'], how='inner')

## Leitura dos datasets com dados relativos a 01/2022 - 12/2022


In [9]:

df_meteo = pd.read_csv("../../datasets/parte2/treino/meteo_202201-202212.csv")
df_energia = pd.read_csv("../../datasets/parte2/treino/energia_202201-202212.csv",na_filter=False)

In [10]:
df_meteo['dt_iso'] = pd.to_datetime(df_meteo['dt_iso'].str.replace(' UTC', ''), format='%Y-%m-%d %H:%M:%S %z')
df_meteo['Data'] = df_meteo['dt_iso'].dt.strftime('%Y-%m-%d')
df_meteo['Hora'] = df_meteo['dt_iso'].dt.strftime('%H')
df_meteo['Hora'] = df_meteo['Hora'].astype(int)

### Eliminar a coluna 'dt_iso' do dataset df_meteo pois os dados relevantes dessa coluna estão na nova coluna 'Data' e 'Hora' 

In [11]:
df_meteo = df_meteo.drop('dt_iso',axis = 1)

### Limpeza das colunas que não acrescentam muita informação relativa ao dataset  

In [12]:
valoresDif = df_meteo['city_name'].value_counts()

In [13]:
df_meteo = df_meteo.drop('city_name', axis = 1)
df_meteo = df_meteo.drop('sea_level', axis = 1) # Como os valores são sempre NaN e não são úteis para o contexto do problema podem ser retirados
df_meteo = df_meteo.drop('grnd_level', axis = 1) # Como os valores são sempre NaN e não são úteis para o contexto do problema podem ser retirados
df_meteo = df_meteo.drop('dt', axis = 1) # Como os valores não acrescentam informação relevante para o conexto do problema podem ser retirados 

### Tratamento de dados do dataset df_energia

### Alteração da coluna 'Hora' do dataset df_energia para o tipo int

In [14]:
df_energia['Hora'] = df_energia['Hora'].astype(int)

### Junção dos datasets df_meteo e df_energia de 09/2021 - 12/2021 para a utilização de Random Forests

In [15]:
df_2022= pd.merge(df_meteo, df_energia, on=['Data','Hora'], how='inner')

## Junção dos datasets com informação de 2021 e 2022

In [16]:
df = pd.concat([df_2021,df_2022],ignore_index = True)

### Criação do label encoding para codificarmos a coluna weather_description

In [17]:
label_encoder = LabelEncoder()
df['weather_description_encoded'] = label_encoder.fit_transform(df['weather_description'])
df = df.drop('weather_description', axis = 1)
df = df.drop('Data', axis = 1)

### Tratamento dos missing values

In [18]:
media_Rain_1h = df['rain_1h'].mean()
df['rain_1h'].fillna(media_Rain_1h, inplace=True)

### Criação da variavel que queremos prever 

In [19]:
x = df.drop('Injeção na rede (kWh)', axis = 1)
y = df['Injeção na rede (kWh)']

### Divisão do dataset em treino e teste 

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.204793028, random_state=42)

### Criação do modelo de random forest, com 100 árvores 

In [21]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

### Treino do modelo 

In [22]:
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

### Previsões do modelo 

In [23]:
y_pred = model.predict(x_test)

### Escrita em ficheiro csv 

In [24]:
df_predicoes = pd.DataFrame({'Result': y_pred})
df_predicoes['RowId'] = range(1, len(y_pred) + 1)
df_predicoes = df_predicoes[['RowId', 'Result']]

df_predicoes.to_csv('../../datasets/parte2/teste/previsoes.csv', index=False)